In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.linear_model import Ridge
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
%time
rawdata0 = pd.read_csv('20170709-pedestrians.csv')
#rawdata = pandas.read_csv('20170709-pedestrians_sample.csv')
rawdata0.tail()

## Data mining

In [ ]:
rawdata0.describe()

#rawdata0.dtypes

In [ ]:
#plt.hist(rawdata0['dx'])
# rawdata0.dtypes
fig, axs = plt.subplots(1,2, figsize=(12,6))
axs[0].hist(rawdata0['dx'], bins=20)
axs[1].hist(rawdata0['dy'], bins=20)
plt.show()
fig, axs = plt.subplots(1,2, figsize=(12,6))
axs[0].hist(rawdata0['ddays'], bins=20)
counts, bins, patches = axs[1].hist(rawdata0['time'], bins=20)
#axs[1].set_xticks(bins)
plt.show()
fig, axs = plt.subplots(1,2, figsize=(12,6))
h = axs[0].hist(rawdata0['dow'], bins=20)
plt.show()
fig = plt.figure(figsize=(12,6))
counts, bins, patches = plt.hist(rawdata0['time'], bins=20)
k = plt.xticks(bins)

In [ ]:
# Filering tails of the histogram in the hour plot
rawdata = rawdata0[(rawdata0['time'] > 7) & (rawdata0['time'] <= 19) ]
rawdata.describe()

In [ ]:
# Normalization
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
ndata = rawdata.copy()
np_scaled = min_max_scaler.fit_transform(ndata[['dx', 'dy', 'ddays', 'time', 'dow']])
ndata[['dx', 'dy', 'ddays', 'time', 'dow']] = np_scaled
ndata.describe()

## Non filtered data

In [ ]:
%time
mydata = rawdata
mod = smf.ols('people ~ dx + dy + ddays + time', data=mydata)
res = mod.fit()
print(res.summary())

## Filtering by Monday

In [ ]:
%time
mydata = rawdata[rawdata.dow == 1]
mod = smf.ols('people ~ dx + dy + ddays + time', data=mydata)
res = mod.fit()
print(res.summary())

## Stepwise Regression
Got the code below from http://planspace.org/20150423-forward_selection_with_statsmodels/

In [ ]:
 def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [ ]:
%time
model = forward_selected(mydata[['dx', 'dy', 'ddays', 'time', 'people']], 'people')
print(model.model.formula)
model.summary()

## Ridge regression

In [ ]:
%time
clf = Ridge(alpha=0.5)
res = clf.fit(mydata[['dx', 'dy', 'ddays', 'time']], mydata[['people']])
res.score(mydata[['dx', 'dy', 'ddays', 'time']], mydata[['people']])

## Scipy off-the-shelf algos

Scipy offers interesting interpolation approaches:
https://docs.scipy.org/doc/scipy/reference/interpolate.html

They provide a set of tools for scattered data, but among them, just two drawed my attention:
* nearest neighbour
* rbf

Matlab also provides some methods for scattered points interpolations, such as `griddatan`.

In [ ]:
from scipy.interpolate import griddata
points = mydata[['dx', 'dy', 'ddays', 'time']].as_matrix()
values = mydata[['people']].as_matrix()
mins = pd.DataFrame.min(mydata[['dx', 'dy', 'ddays', 'time']])
maxs = pd.DataFrame.max(mydata[['dx', 'dy', 'ddays', 'time']])
npartitions = 20
griddx, griddy,gridddays, gridtime = np.mgrid[mins['dx']:maxs['dx']:npartitions,
         mins['dy']:maxs['dy']:npartitions,
         mins['ddays']:maxs['ddays']:npartitions,
         mins['time']:maxs['time']:npartitions,
        ]

#Each segment below takes ~30min in a single-threaded run
#grid = griddata(points, values, (griddx, griddy, gridddays, gridtime), method='nearest')
#np.save('gridnearest.npy', grid)

#Linear interpolation returns just NaN values
#grid = griddata(points, values, (griddx, griddy, gridddays, gridtime), method='linear')
#np.save('gridlinear.npy', grid)

#Gives error. Cubic is for 1d or 2d data
#grid = griddata(points, values, (griddx, griddy, gridddays, gridtime), method='cubic')

#np.save('gridcubic.npy', grid)

In [ ]:
# Generates memory error. Very memory intensive, even using 64GB in the cluster
#from scipy.interpolate import Rbf
#rbfi = Rbf(points[:,0],points[:,1],points[:,2],points[:,3], values)

## SVM RBF Scikit learn

In [ ]:
from sklearn import svm, preprocessing
import pandas
import time

# For a sample of size 100000 it took 608.8498375415802
t0 = time.time()
samplesz = 10000
_data = ndata.sample(n=samplesz)
skpoints = _data[['dx', 'dy', 'ddays', 'time']]
clf = svm.SVR(kernel='rbf', C=1.0, verbose=True, cache_size=20000)
#clf.fit(skpoints, np.array(_data[['people']]).ravel() )
#print(clf.predict(skpoints.sample(n=1)[['dx', 'dy', 'ddays', 'time']]))
#print('For a sample of size {} it took {}'.format(samplesz, time.time() - t0))

In [ ]:
# Partition in train/val and test
from numpy import random
trainratio = 0.8
msk = np.random.rand(len(ndata)) < trainratio
traindata = ndata[msk]
testdata  = ndata[~msk]

In [ ]:
# Here I compute the suppport
scaledddx = 1000.0/(np.max(rawdata['dx']) - np.min(rawdata['dx']))
scaledddy = 1000.0/(np.max(rawdata['dy']) - np.min(rawdata['dy']))
scaledddays = 5.0/326
scaleddtime = 4.0/(np.max(rawdata['time']) - np.min(rawdata['time']))
print(scaledddx, scaledddy, scaledddays, scaleddtime)

In [ ]:
p = 0.6
filtereddata = ndata.copy()
filtereddata = filtereddata[(filtereddata['dx'] < p + scaledddx) & (filtereddata['dx'] > p - scaledddx)]
print(filtereddata['dx'].count())
filtereddata = filtereddata[(filtereddata['dy'] < p + scaledddy) & (filtereddata['dy'] > p - scaledddy)]
print(filtereddata['dx'].count())
filtereddata = filtereddata[(filtereddata['ddays'] < p + scaledddays) & (filtereddata['ddays'] > p - scaledddays)]
print(filtereddata['dx'].count())
filtereddata = filtereddata[(filtereddata['time'] < p + scaleddtime) & (filtereddata['time'] > p - scaleddtime)]
print(filtereddata['dx'].count())

In [ ]:
mins = (np.min(ndata)[['dx', 'dy', 'ddays', 'time']]).as_matrix()
maxs = np.max(ndata)[['dx', 'dy', 'ddays', 'time']].as_matrix()
steps = [scaledddx, scaledddy, scaledddays, scaleddtime]

In [ ]:
from itertools import product
mygrid = list(product(*[np.arange(i, j, k)[:-1] for i,j,k in zip(mins, maxs, steps)]))
acc = 0
for p in mygrid:
#     print('###############################################')
#     print(p)
    filtereddata = traindata.copy()
    filtereddata = filtereddata[(filtereddata['dx'] < p[0] + scaledddx) & (filtereddata['dx'] > p[0] - scaledddx)]
#     print(len(filtereddata.index))
    filtereddata = filtereddata[(filtereddata['dy'] < p[1] + scaledddy) & (filtereddata['dy'] > p[1] - scaledddy)]
#     print(len(filtereddata.index))
    filtereddata = filtereddata[(filtereddata['ddays'] < p[2] + scaledddays) & (filtereddata['ddays'] > p[2] - scaledddays)]
#     print(len(filtereddata.index))
    filtereddata = filtereddata[(filtereddata['time'] < p[3] + scaleddtime) & (filtereddata['time'] > p[3] - scaleddtime)]
    sz = len(filtereddata.index)
    
    if sz > 3:
        print(p)
        print(acc, sz)
        acc += 1        

In [ ]:
print(len(traindata.index))